In [2]:
# Instalamos requerimientos
!pip install -r /content/drive/MyDrive/basurilla/requirements_bert.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


IMPORTACION DE LIBRERIAS:|
--------------------------

- data_preprocessing es un script donde se han agrupado las funciones especificas desarrolladas para el preprocesado de los datos originales que ha permitido agilizar el proceso iterativo de pruebas consecutivas:
    - load_data: Montar GDrive si es necesario + carga de csv + busqueda de duplicados, valores nulos y su eliminación si asi se decide
    - data_balance: balancear el numero de datos por clase de diagnóstico
    - min_tweets_filter: filtrar por numero de tweets
    - clean_tweets: Elimina de los tweets residuos del preprocesado y anonimización llevado a cabo por los investigadores del instituto tecnologico de monterrey.
    - group_tweets: agrupar tweets en grupos de n para que el modelo tenga un mayor contexto semantico
    - data_split_for_training: Split de datos para evaluacion de modelos

- Ademas de las habituales para manejo de datos como Pandas y numpy, las metricas habituales y el split de datos, se han de importar también las bibliotecas especificas de pytorch, transformers de hugginface, optuna, clases predefinidas para el manejo de los datos, modulo para gestion de la memoria GPU, serialización de computos intermedios y gestores para el entrenamiento del modelo.

In [13]:
# Manejo de datos
import pandas as pd
import numpy as np


# Funciones especificas de preprocesado de los datos
import preprocessing_functions2 as pf


import sklearn
# Split de datos para modelos de evaluación
from sklearn.model_selection import train_test_split
# Mtetricas de evaluacion
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# PyTorch y manejo de GPU
import torch

from torch.utils.data import (
    DataLoader,         # Clase que proporciona iteradores sobre un conjunto de datos.
    TensorDataset,      # Conjunto de datos simple que envuelve tensores.
    RandomSampler,      # Sampler que obtiene elementos de un conjunto de datos de manera aleatoria.
    SequentialSampler   # Sampler que obtiene elementos de un conjunto de datos de manera secuencial.
)

import gc  # limpiar memoria GPU

# Transformers de Hugging Face
import transformers
from transformers import (
    BertForSequenceClassification,  # Modelo preentrenado de BERT para clasificación de secuencias.
    BertModel,                      # Modelo base de BERT sin una cabeza de clasificación específica.
    BertTokenizer,                  # Tokenizador de texto específico de BERT.
    Trainer,                        # Módulo para gestionar y entrenar modelos de Transformers.
    TrainingArguments,              # Clase para definir los argumentos de entrenamiento.
    BertConfig,                     # Clase de configuración para los modelos BERT.
    EarlyStoppingCallback           # Callback para detener el entrenamiento temprano si no hay mejoras.
)


# Mostrar información periodica sobre la evolución del entrenamiento del modelo
transformers.logging.set_verbosity_info()

# Optuna para optimización de hiperparámetros
import optuna

# Optimización de entrenamiento
from accelerate import Accelerator

# Serialización de modelos y datos
import pickle

# Configura pandas para usar el modo copy-on-write
pd.options.mode.copy_on_write = True


Importar datos:|
---------------
#
- Funcion load_data
#
Balanceo de clases:|
---------------------
#
- Las mejores metricas se han encontrado balanceando los datos sobre el maximo de la segunda clase de diagnosticado mayoritaria "DEPRESSION".

- Se ajusta a la baja la clase "AHDH" a 200000 muestras y el resto de clases permanecen igual.
#
Minimo de tweets:|
-----------------
#
- Se filtra el Dataframe para que los usuarios tengan un minimo de 40 tweets.

- Este ajuste es recomendable teniendo en cuenta que los tweets se agruparan por usuario, de 20 en 20 o hasta completar el total de tweets de cada usuario
#

In [ ]:
# Definimos la ruta donde se encuentra el CSV
file_path = '/content/drive/MyDrive/basurilla/df_concat_english.csv'

# Llamamos a la funcion para crear el dataframe y gestionar duplicados y nulos
df = pf.load_data(file_path)

# Llamado a la funcion para el balanceado de clases
df_balanced = pf.data_balance(df, 'class', 'CONTROL', 1000000, 225000)

# Llamada a la funcion para filtrar por minimo de tweets por usuario
df_filtered = pf.min_tweets_filter(df_balanced, 40)

# Llamada a la funcion para limpieza del texto de los tweets
df_clean = pf.clean_tweets(df_filtered, column='tweet')


MAPEADO DE CLASES BINARIO|
-------------------------

In [16]:
# Se aplica una funcion para convertir la clase control y el resto de clases con usuarios diagnosticados en una salida binaria
df_clean['diagnosed'] = df_clean['class'].map(lambda x: 0 if x=='control' else 1)

In [ ]:

# Llamada a la funcion para extraer un porcentaje de los datos para la busqueda de hiperparametros
df_training = pf.data_split_for_training(df_clean, 0.001, 'class')

Definimos las columnas necesarias para el modelo final|
--------------------------------------------------------

In [27]:
df_final = df_training[['tweet', 'diagnosed']]

Dataframe final|
---------------

In [ ]:
# Ultima revision del dataframe final
print(df_final['diagnosed'].value_counts())
print(df_final.isna().sum())
print(df_final.duplicated().sum())

----------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------------

TOKENIZACION|
------------


In [ ]:
# Dividir los datos en entrenamiento, validación y test
X_train, X_temp, y_train, y_temp = train_test_split(df_final, df_final['diagnosed'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Crear instancia de BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizar los tweets
train_encodings = tokenizer(X_train['tweet'].tolist(), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(X_val['tweet'].tolist(), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(X_test['tweet'].tolist(), truncation=True, padding=True, return_tensors="pt")


CREACION DE DATASETS|
--------------------

In [29]:
# Definición de la Clase TweetDataset que hereda de torch.utils.data.Dataset para manejar los datos de tweets y sus etiquetas.
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    # __getitem__ toma los datos de los encodings y labels y devuelve un paquete con cada [tweet - etiqueta]
    def __getitem__(self, idx):
        # Capturar uno a uno los encodings de los tweets
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Etiquetas de salida
        item['labels'] = self.labels[idx]
        # Devolver esos datos empaquetados
        return item

    def __len__(self):
        return len(self.labels)

#  Crear instancias de esta clase para los conjuntos de datos de entrenamiento, validación y prueba
train_dataset = TweetDataset(train_encodings, y_train.tolist())
val_dataset = TweetDataset(val_encodings, y_val.tolist())
test_dataset = TweetDataset(test_encodings, y_test.tolist())

In [ ]:
# Exportar encodings de BERT

# Encodings de los tweets
with open('/content/drive/MyDrive/Colab Notebooks/train_encodings.pkl', 'wb') as f:
    pickle.dump(train_encodings, f)
with open('/content/drive/MyDrive/Colab Notebooks/val_encodings.pkl', 'wb') as f:
    pickle.dump(val_encodings, f)
with open('/content/drive/MyDrive/Colab Notebooks/test_encodings.pkl', 'wb') as f:
    pickle.dump(test_encodings, f)

# Etiquetas verdaderas
with open('/content/drive/MyDrive/Colab Notebooks/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('/content/drive/MyDrive/Colab Notebooks/y_val.pkl', 'wb') as f:
    pickle.dump(y_val, f)
with open('/content/drive/MyDrive/Colab Notebooks/y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)


# Datasets finales
with open('/content/drive/MyDrive/Colab Notebooks/train_dataset.pkl', 'wb') as f:
    pickle.dump(train_dataset, f)
with open('/content/drive/MyDrive/Colab Notebooks/val_dataset.pkl', 'wb') as f:
    pickle.dump(val_dataset, f)
with open('/content/drive/MyDrive/Colab Notebooks/test_dataset_.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)

GESTION DEL DISPOSITIVO DE COMPUTO|
----------------------------------

In [23]:
# Definicion del dispositivo a usar segun disponibilidad (preferentemente CUDA)

import torch

if torch.cuda.is_available():
    print("CUDA disponible. Entrenando en GPU.")
    device = torch.device("cuda")
else:
    print("CUDA no disponible. Entrenando en CPU.")
    device = torch.device("cpu")


CUDA no disponible. Entrenando en CPU.


In [24]:
# limpiar memoria gpu

gc.collect()           # Recolecta basura en el nivel de Python
torch.cuda.empty_cache()  # Limpia la caché de CUDA

MODELO: DEFINIR, EJECUTAR Y MOSTRAR OPTIMIZACIÓN |
---------------------------------------------------

In [ ]:
# Definimnos función para calcular métricas durante la evaluación
def compute_metrics(pred):
    print("Computing metrics...")  # Confirma que la función ha sido llamada
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, pred.predictions[:, 1])
    print(f"Computed accuracy: {acc}")  # mostrar resultado de accuracy
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc
    }

# Definicion de la función objetivo de optuna
def objective(trial):
    # Instanciamos Bertconfig para poder definir algunos parametros propios del modelo
    config = BertConfig.from_pretrained('bert-base-uncased')
    config.num_labels = 2

    # Espacio de hiperparametros para optimizar
    config.hidden_dropout_prob = trial.suggest_float("hidden_dropout_prob", 0.1, 0.5)                   # Probabilidad de droptout en las capas ocultas
    config.attention_probs_dropout_prob = trial.suggest_float("attention_probs_dropout_prob", 0.1, 0.5) # Probabilidad de droptout del mecanismo de atención
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.1)
    warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    eval_steps = trial.suggest_int('eval_steps', 500, 5000)

    # Creamos instancia del modelo
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
    # Enviamos modelo a la GPU si esta disponible
    model = model.to(device)


    # Argumentos para el trainer
    output_dir = f'./results/trial_{trial.number}'
    training_args = TrainingArguments(
        output_dir=output_dir,                       # directorio de salida para los resultados del modelo
        num_train_epochs=3,                          # número total de épocas de entrenamiento
        evaluation_strategy='steps',                 # estrategia de evaluación periodica
        eval_steps = eval_steps,                     # pasos completados hasta la evaluación periodica
        save_steps= eval_steps,                      # pasos completados hasta el guardado de un checkpoint
        per_device_train_batch_size=batch_size,      # tamaño del batch para entrenamiento
        per_device_eval_batch_size=batch_size,       # tamaño del batch para evaluación
        warmup_steps=warmup_steps,                   # pasos hasta el incremento maximo de learning rate al inicio
        weight_decay=weight_decay,                   # fuerza de la decay del peso
        logging_dir=f'./logs/trial_{trial.number}',  # directorio para almacenar logs
        load_best_model_at_end=True,                 # guardar un checkpoint al final de cada época
        logging_steps=10,                            # número máximo de checkpoints a guardar
        save_strategy='steps',                       # estrategia de checkpoint
        report_to='none'
    )




    # Inicializar el Trainer con los argumentos de entrenamiento
    early_stopping = EarlyStoppingCallback(early_stopping_patience=7, early_stopping_threshold=0.01)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    print(eval_results)

    loss = eval_results.get("eval_loss")
    if loss is None:
        raise ValueError("Accuracy could not be computed.")

    return loss


# Creación del estudio de optuna
study = optuna.create_study(direction='minimize')
# Llamada a la instancia recien creada de optuna definiendo funcion objetivo y numero de pruebas
study.optimize(objective, n_trials=10)


# Una vez completado el estudio, obtener el mejor ensayo
best_trial = study.best_trial

# Muestra los mejores parámetros
print("Mejores parámetros: ", best_trial.params)

# Mejor valor de la función objetivo
print("Mejor valor (pérdida mínima):", best_trial.value)


ENTRENAMIENTO FINAL CON LOS MEJORES HIPERPARAMETROS ENCONTRADOS|
------------------------------------------------------------------

In [ ]:
#Configuracion del modelo
config = BertConfig.from_pretrained('bert-base-uncased')
config.hidden_dropout_prob = best_trial.params['hidden_dropout_prob']
config.attention_probs_dropout_prob = best_trial.params['attention_probs_dropout_prob']
config.num_labels = 2
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model = model.to(device)

# Funcion de métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, pred.predictions[:, 1])
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auc': auc
    }

output_dir = '/content/drive/MyDrive/Colab Notebooks/proyecto final bootcamp/berts/bert_model_balanced_final/training_checkpoints'
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    evaluation_strategy='steps',
    learning_rate=best_trial.params['learning_rate'],
    eval_steps=2000,
    save_steps=2000,
    per_device_train_batch_size=best_trial.params['batch_size'],
    per_device_eval_batch_size=best_trial.params['batch_size'],
    warmup_steps=best_trial.params['warmup_steps'],
    weight_decay=best_trial.params['weight_decay'],
    load_best_model_at_end=True,
    save_strategy='steps'
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=7, early_stopping_threshold=0.01)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Inicializar entrenamiento
trainer.train()


EVALUACIÓN DE LOS RESULTADOS DEL ENTRENAMIENTO|
--------------------------------------------

In [ ]:

# Evaluar el modelo en el conjunto de entrenamiento
train_metrics = trainer.evaluate(train_dataset)
print("Train Metrics:", train_metrics)

# Evaluar el modelo en el conjunto de validación
val_metrics = trainer.evaluate(val_dataset)
print("Validation Metrics:", val_metrics)

In [ ]:
# Obtener las predicciones en el conjunto de datos de prueba
predictions = trainer.predict(test_dataset)

# Calcular las métricas utilizando las predicciones y las etiquetas verdaderas
metrics = compute_metrics(predictions)

# Imprimir las métricas
print("Accuracy:", metrics['accuracy'])
print("Precision:", metrics['precision'])
print("Recall:", metrics['recall'])
print("F1 Score:", metrics['f1'])
print("AUC:", metrics['auc'])

---------------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Guardar el modelo final
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks')
